In [ ]:
project_path = "/home/jupyter"
import os
import sys
sys.path.append(project_path)
from google.cloud import bigquery, storage

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px

from fintrans_toolbox.src import bq_utils as bq

client = bigquery.Client()

In [ ]:
# Summarise the data by UK Cardholder Spending All Quarterly --------------- Cardholders' Number Total Quarterly ----  mcg = 'All'

UK_spending_by_mcg_All = '''SELECT time_period_value, cardholders, destination_country, spend 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and mcg = 'All' 
and merchant_channel = 'All' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
 
GROUP BY cardholders, destination_country, 
time_period_value, spend 
ORDER BY time_period_value, destination_country DESC'''
df_by_mcg_All = bq.read_bq_table_sql(client, UK_spending_by_mcg_All)
df_by_mcg_All.head()

# Caculate UK Domestic Total Spending Quarterly

# Assuming df_by_mcg_All is the DataFrame returned from the BigQuery query
# Then group by 'time_period_value' and sum the 'spend' for each quarter

# Check if df_by_mcg_All is not None and has the expected columns
if df_by_mcg_All is not None and 'time_period_value' in df_by_mcg_All.columns and 'spend' in df_by_mcg_All.columns:
    # Group by quarter and sum the spend
    UK_spending_by_mcg_All = df_by_mcg_All.groupby('time_period_value')['cardholders'].sum().reset_index()
   
 # Rename the column
    UK_spending_by_mcg_All = UK_spending_by_mcg_All.rename(columns={'cardholders': 'Spend_mcg_All_cardholders'})
    print(UK_spending_by_mcg_All)
else:
    print("DataFrame is empty or missing required columns.")

In [ ]:
# Total UK Cardholder Total Quarterly

# Summarise the data by UK Cardholder Online Spending Total Quarterly

UK_spending_UK_All = '''SELECT time_period_value, SUM(spend) AS total_spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and mcg = 'All'
and merchant_channel = 'All' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
GROUP BY 
time_period_value 
ORDER BY time_period_value'''
df_by_UK_All = bq.read_bq_table_sql(client, UK_spending_UK_All)
df_by_UK_All = df_by_UK_All.rename(columns={'total_spend': 'UK_spend_All'})
df_by_UK_All.head()

print(df_by_UK_All)

# Save the DataFrame to a CSV file
csv_filename = "UK_All_Spending.csv"
df_by_UK_All.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been created successfully.")

In [ ]:
# Total UK Cardholder Online Quarterly

# Summarise the data by UK Cardholder Online Spending Total Quarterly

UK_spending_Online_All = '''SELECT time_period_value, SUM(spend) AS total_spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and mcg = 'All'
and merchant_channel = 'Online' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
GROUP BY 
time_period_value 
ORDER BY time_period_value'''
df_by_Online_All = bq.read_bq_table_sql(client, UK_spending_Online_All)
df_by_Online_All = df_by_Online_All.rename(columns={'total_spend': 'Online_spend_All'})
df_by_Online_All.head()

print(df_by_Online_All)

# Save the DataFrame to a CSV file
csv_filename = "UK_Online_Spending.csv"
df_by_Online_All.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been created successfully.")

In [ ]:
# Online Quarterly Ratio

import pandas as pd

# Load the UK online and all spending data
online_spending = pd.read_csv("UK_Online_Spending.csv")
all_spending = pd.read_csv("UK_All_Spending.csv")

# Merge the two datasets on the common time column
merged_df = pd.merge(online_spending, all_spending, on='time_period_value')

# Calculate the online spending ratio
merged_df['online_spending_ratio'] = (merged_df['Online_spend_All'] / merged_df['UK_spend_All']) * 100

# Save the result to a new CSV file
merged_df.to_csv("UK_Online_Spending_Ratio.csv", index=False)

# Display the first few rows of the result
merged_df.head(25)



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the UK online spending ratio data
file_path = "UK_Online_Spending_Ratio.csv"
df = pd.read_csv(file_path)

# Ensure the time column is treated as categorical or datetime for proper plotting
if 'time_period_value' in df.columns:
    time_col = 'time_period_value'
elif 'Quarter' in df.columns:
    time_col = 'Quarter'
else:
    time_col = df.columns[0]  # fallback to first column

# Plot the line chart
plt.figure(figsize=(10, 6))
plt.plot(df[time_col], df['online_spending_ratio'], marker='o', linestyle='-')
plt.title('UK Quarterly Online Spending Ratio')
plt.xlabel('Quarter')
plt.ylabel('Online Spending Ratio (%)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()

# Save the chart
plt.savefig("UK_Online_Spending_Ratio_Chart.png")
print("Line chart saved as UK_Online_Spending_Ratio_Chart.png")



In [ ]:
# Quaterly Online Spend Adjusted Quarterly Value --------------------------- --- mcg = 'All'

# Fromula : Adjust_Quarterly_Online_Spend = ( Online_spend_All_cardholders (2019Q1) / Spend_mcg_All_cardholders (quarter)) × Online_spend_All (quarter)

import pandas as pd

# Load the two datasets
df_cardholders = pd.read_csv("UK_spending_by_mcg_All.csv")
df_all = pd.read_csv("UK_Online_Spending.csv")

# Get 2019Q1 values
cardholders_2019Q1 = df_cardholders.loc[df_cardholders["time_period_value"] == "2019Q1", "Spend_mcg_All_cardholders"].values[0]

# Calculate Adjusted Quarterly Online Spend
df_cardholders["Adjust_Quarterly_Online_Spend"] = (
    cardholders_2019Q1 / df_cardholders["Spend_mcg_All_cardholders"] 
) * df_all["Online_spend_All"]

# Save or view the result
df_cardholders.to_csv("Adjusted_Online_Spend_mcg.csv", index=False)
print(df_cardholders)


In [ ]:
# Total UK Cardholder Face to Face Quarterly

# Summarise the data by UK Cardholder F2F Spending Total Quarterly

UK_spending_F2F_All = '''SELECT time_period_value, SUM(spend) AS total_spend
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and mcg = 'All'
and merchant_channel = 'Face to Face' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
GROUP BY 
time_period_value 
ORDER BY time_period_value'''
df_by_F2F_All = bq.read_bq_table_sql(client, UK_spending_F2F_All)
df_by_F2F_All = df_by_F2F_All.rename(columns={'total_spend': 'F2F_spend_All'})
df_by_F2F_All.head()

print(df_by_F2F_All)

# Save the DataFrame to a CSV file
csv_filename = "UK_F2F_Spending.csv"
df_by_F2F_All.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been created successfully.")

In [ ]:
# Quaterly Face to Face Spend Adjusted Quarterly Value --------------------------- --- mcg = 'All'

# Fromula : Adjust_Quarterly_F2F_Spend = ( F2F_spend_All_cardholders (2019Q1) / Spend_mcg_All_cardholders (quarter)) × Online_spend_All (quarter)

import pandas as pd

# Load the two datasets
df_cardholders = pd.read_csv("UK_spending_by_mcg_All.csv")
df_all = pd.read_csv("UK_F2F_Spending.csv")

# Get 2019Q1 values
cardholders_2019Q1 = df_cardholders.loc[df_cardholders["time_period_value"] == "2019Q1", "Spend_mcg_All_cardholders"].values[0]

# Calculate Adjusted Quarterly Online Spend
df_cardholders["Adjust_Quarterly_F2F_Spend"] = (
    cardholders_2019Q1 / df_cardholders["Spend_mcg_All_cardholders"] 
) * df_all["F2F_spend_All"]

# Save or view the result
df_cardholders.to_csv("Adjusted_F2F_Spend_mcg.csv", index=False)
print(df_cardholders)


In [ ]:
# Compare Total Online and F2F totals with 2019 ave as base value = 100

import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV files
online_df = pd.read_csv("UK_Online_Spending.csv")
f2f_df = pd.read_csv("UK_F2F_Spending.csv")

# Ensure the time_period_value column is sorted and consistent
online_df = online_df.sort_values("time_period_value")
f2f_df = f2f_df.sort_values("time_period_value")

# Merge the two datasets on time_period_value
merged_df = pd.merge(f2f_df, online_df, on="time_period_value", how="inner")

# Calculate 2019 average for both series
online_2019_avg = merged_df[merged_df["time_period_value"].str.startswith("2019")]["Online_spend_All"].mean()
f2f_2019_avg = merged_df[merged_df["time_period_value"].str.startswith("2019")]["F2F_spend_All"].mean()

# Normalize both series to 2019 average = 100
merged_df["Online_index"] = (merged_df["Online_spend_All"] / online_2019_avg) * 100
merged_df["F2F_index"] = (merged_df["F2F_spend_All"] / f2f_2019_avg) * 100

# Adjust both series so they start from 100 in 2019Q1
online_start = merged_df.loc[merged_df["time_period_value"] == "2019Q1", "Online_index"].values[0]
f2f_start = merged_df.loc[merged_df["time_period_value"] == "2019Q1", "F2F_index"].values[0]
merged_df["Online_index"] = merged_df["Online_index"] * (100 / online_start)
merged_df["F2F_index"] = merged_df["F2F_index"] * (100 / f2f_start)

# Plot the indexed trends
plt.figure(figsize=(12, 6))
plt.plot(merged_df["time_period_value"], merged_df["Online_index"], label="Online Spending Index", marker='o')
plt.plot(merged_df["time_period_value"], merged_df["F2F_index"], label="Face-to-Face Spending Index", marker='o')
plt.xticks(rotation=45)
plt.xlabel("Quarter")
plt.ylabel("Index (2019 Average = 100)")
plt.title("UK Total Spending Trends (Indexed to 2019 Average = 100)")
plt.legend()
plt.tight_layout()
plt.grid(True)
plt.savefig("UK_Total_Spending_Index_Chart.png")
plt.show()


In [ ]:
# Indexed Spend F2F All
# Indexed card spending data (average 2019 equals 100) is calculated :
# Spend=(Quarter Spend / Average Spend in 2019) × 100
#Indexed Spend = (F2F_spend_All / Average F2F Spend in 2019 (which is 2019Q1 - 2019Q4 ave)) × 100

import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv("UK_F2F_Spending.csv")

# Filter rows for 2019Q1 to 2019Q4
df_2019 = df[df['time_period_value'].isin(["2019Q1", "2019Q2", "2019Q3", "2019Q4"])]

# Calculate the average adjusted_F2F_spend for 2019
base_avg = df_2019['F2F_spend_All'].mean()

# Compute Indexed Spend
df['Indexed Spend F2F All'] = (df['F2F_spend_All'] / base_avg) * 100

# Plot the Indexed Spend over time
plt.figure(figsize=(12, 6))
plt.plot(df['time_period_value'], df['Indexed Spend F2F All'], marker='o', linestyle='-')
plt.title('Indexed All F2F Spend Over Time (Base: 2019 Average = 100)')
plt.xlabel('Time Period')
plt.ylabel('Indexed Spend')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig("indexed_spend_plot.png")
plt.show()


# Save to CSV
df.to_csv("Indexed_All_F2F.csv", index=False)

# Output the path to the saved file
"Indexed_All_F2F.csv"


In [ ]:
# Indexed Spend F2F All  ---------------- mcg = 'All'
# Indexed card spending data (average 2019 equals 100) is calculated :
# Spend=(Quarter Spend / Average Spend in 2019) × 100
#Indexed Spend = (F2F_spend_All / Average F2F Spend in 2019 (which is 2019Q1 - 2019Q4 ave)) × 100

import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv("Adjusted_F2F_Spend_mcg.csv")

# Filter rows for 2019Q1 to 2019Q4
df_2019 = df[df['time_period_value'].isin(["2019Q1", "2019Q2", "2019Q3", "2019Q4"])]

# Calculate the average adjusted_F2F_spend for 2019
base_avg = df_2019['Adjust_Quarterly_F2F_Spend'].mean()

# Compute Indexed Spend
df['Indexed Spend F2F All'] = (df['Adjust_Quarterly_F2F_Spend'] / base_avg) * 100

# Plot the Indexed Spend over time
plt.figure(figsize=(12, 6))
plt.plot(df['time_period_value'], df['Indexed Spend F2F All'], marker='o', linestyle='-')
plt.title('Indexed All F2F Spend Over Time (Base: 2019 Average = 100)')
plt.xlabel('Time Period')
plt.ylabel('Indexed Spend')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig("indexed_spend_plot.png")
plt.show()


# Save to CSV
df.to_csv("Indexed_All_F2F_mcg.csv", index=False)

# Output the path to the saved file
"Indexed_All_F2F_mcg.csv"


In [ ]:
# Indexed Spend Online All
# Indexed card spending data (average 2019 equals 100) is calculated :
# Spend=(Quarter Spend / Average Spend in 2019) × 100
#Indexed Spend = (Online_spend_All / Average Online Spend in 2019 (which is 2019Q1 - 2019Q4 ave)) × 100

import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv("UK_Online_Spending.csv")

# Filter rows for 2019Q1 to 2019Q4
df_2019 = df[df['time_period_value'].isin(["2019Q1", "2019Q2", "2019Q3", "2019Q4"])]

# Calculate the average adjusted_F2F_spend for 2019
base_avg = df_2019['Online_spend_All'].mean()

# Compute Indexed Spend
df['Indexed Spend Online All'] = (df['Online_spend_All'] / base_avg) * 100

# Plot the Indexed Spend over time
plt.figure(figsize=(12, 6))
plt.plot(df['time_period_value'], df['Indexed Spend Online All'], marker='o', linestyle='-')
plt.title('Indexed All Online Spend Over Time (Base: 2019 Average = 100)')
plt.xlabel('Time Period')
plt.ylabel('Indexed Spend')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig("indexed_spend_plot.png")
plt.show()


# Save to CSV
df.to_csv("Indexed_All_Online.csv", index=False)

# Output the path to the saved file
"Indexed_All_Online.csv"


In [ ]:
# Indexed Spend Online All ----------- mcg = 'All'
# Indexed card spending data (average 2019 equals 100) is calculated :
# Spend=(Quarter Spend / Average Spend in 2019) × 100
#Indexed Spend = (Online_spend_All / Average Online Spend in 2019 (which is 2019Q1 - 2019Q4 ave)) × 100

import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv("Adjusted_Online_Spend_mcg.csv")

# Filter rows for 2019Q1 to 2019Q4
df_2019 = df[df['time_period_value'].isin(["2019Q1", "2019Q2", "2019Q3", "2019Q4"])]

# Calculate the average adjusted_F2F_spend for 2019
base_avg = df_2019['Adjust_Quarterly_Online_Spend'].mean()

# Compute Indexed Spend
df['Indexed Spend Online All'] = (df['Adjust_Quarterly_Online_Spend'] / base_avg) * 100

# Plot the Indexed Spend over time
plt.figure(figsize=(12, 6))
plt.plot(df['time_period_value'], df['Indexed Spend Online All'], marker='o', linestyle='-')
plt.title('Indexed All Online Spend Over Time (Base: 2019 Average = 100)')
plt.xlabel('Time Period')
plt.ylabel('Indexed Spend')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig("indexed_spend_plot.png")
plt.show()


# Save to CSV
df.to_csv("Indexed_All_Online_mcg.csv", index=False)

# Output the path to the saved file
"Indexed_All_Online_mcg.csv"


In [ ]:
# Indexed Line Chart for UK Online vs F2F Totals - Indexed Value - Without Adjust

import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV files
online_df = pd.read_csv("Indexed_All_Online.csv")
f2f_df = pd.read_csv("Indexed_All_F2F.csv")

# Plot the line chart
plt.figure(figsize=(12, 6))
plt.plot(online_df['time_period_value'], online_df['Indexed Spend Online All'], label='Online All', marker='o')
plt.plot(f2f_df['time_period_value'], f2f_df['Indexed Spend F2F All'], label='F2F All', marker='s')

# Customize the plot
plt.title('Indexed Spend Comparison: Online vs F2F All')
plt.xlabel('Time Period')
plt.ylabel('Indexed Spend (Base 2019 = 100)')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()

# Save the plot
plt.savefig("indexed_spend_comparison.png")
plt.show()


In [ ]:
# Indexed Line Chart for UK Online vs F2F Totals - Indexed Value  ---- Without Adjust

import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV files
online_df = pd.read_csv("Indexed_All_Online.csv")
f2f_df = pd.read_csv("Indexed_All_F2F.csv")

# Plot the line chart
plt.figure(figsize=(12, 6))
plt.plot(online_df['time_period_value'], online_df['Indexed Spend Online All'], label='Online All', marker='o')
plt.plot(f2f_df['time_period_value'], f2f_df['Indexed Spend F2F All'], label='F2F All', marker='s')
plt.axvline(x="2023Q2", color='gray', linestyle='--', label="June 2023")

# Customize the plot
plt.title('Indexed Spend Comparison: Online vs F2F All')
plt.xlabel('Time Period')
plt.ylabel('Indexed Spend (Base 2019 = 100)')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()

# Save the plot
plt.savefig("indexed_spend_comparison.png")
plt.show()


In [ ]:
# Indexed Line Chart for UK Online vs F2F Totals - Indexed Value  ---- Adjust mcg = 'All'

import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV files
online_df = pd.read_csv("Indexed_All_Online_mcg.csv")
f2f_df = pd.read_csv("Indexed_All_F2F_mcg.csv")

# Plot the line chart
plt.figure(figsize=(12, 6))
plt.plot(online_df['time_period_value'], online_df['Indexed Spend Online All'], label='Online All', marker='o')
plt.plot(f2f_df['time_period_value'], f2f_df['Indexed Spend F2F All'], label='F2F All', marker='s')
plt.axvline(x="2023Q2", color='gray', linestyle='--', label="June 2023")

# Customize the plot
plt.title('Indexed Spend Comparison: Online vs F2F All')
plt.xlabel('Time Period')
plt.ylabel('Indexed Spend (Base 2019 = 100)')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()

# Save the plot
plt.savefig("indexed_spend_comparison.png")
plt.show()


In [ ]:
# Summarise the data by UK Cardholder Spending All ---- Quarterly

UK_spending_All_Quarter = '''SELECT time_period_value, SUM(spend) AS total_spend 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and mcc = 'All' 
and mcg = 'All' 
and merchant_channel = 'All' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM'  
GROUP BY
time_period_value 
ORDER BY time_period_value'''
df_by_All = bq.read_bq_table_sql(client, UK_spending_All_Quarter)
df_by_All.head()

print(df_by_All)

df_by_All.to_csv('UK_spending_All_Quarter.csv')

In [ ]:
# Quaterly All Spend Adjusted Quarterly Value --------------------------- --- mcg = 'All'

# Fromula : Adjust_Quarterly_All_Spend = ( UK_spend_All_cardholders (2019Q1) / Spend_mcg_All_cardholders (quarter)) × UK_spend_All (quarter)

import pandas as pd

# Load the two datasets
df_cardholders = pd.read_csv("UK_spending_by_mcg_All.csv")
df_all = pd.read_csv("UK_spending_All_Quarter.csv")

# Get 2019Q1 values
cardholders_2019Q1 = df_cardholders.loc[df_cardholders["time_period_value"] == "2019Q1", "Spend_mcg_All_cardholders"].values[0]

# Calculate Adjusted Quarterly Online Spend
df_cardholders["Adjust_Quarterly_All_Spend"] = (
    cardholders_2019Q1 / df_cardholders["Spend_mcg_All_cardholders"] 
) * df_all["total_spend"]

# Save or view the result
df_cardholders.to_csv("Adjusted_UK_spending_All_mcg.csv", index=False)
print(df_cardholders)


In [ ]:
# Indexed Spend UK All ----------- mcg = 'All'
# Indexed card spending data (average 2019 equals 100) is calculated :
# Spend=(Quarter Spend / Average Spend in 2019) × 100
#Indexed Spend = (UK_spend_All / Average UK Spend in 2019 (which is 2019Q1 - 2019Q4 ave)) × 100

import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv("Adjusted_UK_spending_All_mcg.csv")

# Filter rows for 2019Q1 to 2019Q4
df_2019 = df[df['time_period_value'].isin(["2019Q1", "2019Q2", "2019Q3", "2019Q4"])]

# Calculate the average adjusted_F2F_spend for 2019
base_avg = df_2019['Adjust_Quarterly_All_Spend'].mean()

# Compute Indexed Spend
df['Indexed Spend All'] = (df['Adjust_Quarterly_All_Spend'] / base_avg) * 100

# Plot the Indexed Spend over time
plt.figure(figsize=(12, 6))
plt.plot(df['time_period_value'], df['Indexed Spend All'], marker='o', linestyle='-')
plt.title('Indexed All Spend Over Time (Base: 2019 Average = 100)')
plt.xlabel('Time Period')
plt.ylabel('Indexed Spend')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig("indexed_spend_plot.png")
plt.show()


# Save to CSV
df.to_csv("Indexed_All_Spend_mcg.csv", index=False)

# Output the path to the saved file
"Indexed_All_Spend_mcg.csv"

In [ ]:
# Indexed Line Chart for UK All, Online vs F2F Totals - Indexed Value  ---- Adjust mcg = 'All'

import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV files
all_df = pd.read_csv("Indexed_All_Spend_mcg.csv")
online_df = pd.read_csv("Indexed_All_Online_mcg.csv")
f2f_df = pd.read_csv("Indexed_All_F2F_mcg.csv")

# Plot the line chart
plt.figure(figsize=(12, 6))
plt.plot(online_df['time_period_value'], all_df['Indexed Spend All'], label='Spend All', marker='o')
plt.plot(online_df['time_period_value'], online_df['Indexed Spend Online All'], label='Online All', marker='o')
plt.plot(f2f_df['time_period_value'], f2f_df['Indexed Spend F2F All'], label='F2F All', marker='s')
plt.axvline(x="2023Q2", color='gray', linestyle='--', label="June 2023")

# Customize the plot
plt.title('Indexed Spend Comparison:  All vs Online vs F2F')
plt.xlabel('Time Period')
plt.ylabel('Indexed Spend (Base 2019 = 100)')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()

# Save the plot
plt.savefig("indexed_spend_comparison1.png")
plt.show()


In [ ]:
# Three typical 'Diamond' Periods for Up & Downs 
# Indexed Line Chart for UK All, Online vs F2F Totals - Indexed Value  ---- Adjust mcg = 'All'

import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV files
all_df = pd.read_csv("Indexed_All_Spend_mcg.csv")
online_df = pd.read_csv("Indexed_All_Online_mcg.csv")
f2f_df = pd.read_csv("Indexed_All_F2F_mcg.csv")

# Extract the quarters and spend values
quarters = online_df['time_period_value'].tolist()
online_spend = online_df['Indexed Spend Online All'].tolist()
face_to_face_spend = f2f_df['Indexed Spend F2F All'].tolist()
all_spend = all_df['Indexed Spend All'].tolist()

# Plot the line chart
plt.figure(figsize=(12, 6))
plt.plot(quarters, all_spend, label='Spend All', marker='o')
plt.plot(quarters, online_spend, label='Online All', marker='o')
plt.plot(quarters, face_to_face_spend, label='F2F All', marker='s')
plt.axvline(x="2021Q4", color='gray', linestyle='--', label="Q4 2021")
plt.axvline(x="2024Q2", color='gray', linestyle='--', label="Q2 2024")

# Highlight Q1s
highlight_quarters = ["2022Q1", "2023Q1", "2024Q1"]
for q in highlight_quarters:
    if q in quarters:
        idx = quarters.index(q)
        plt.scatter(q, online_spend[idx], color='red', zorder=5)
        plt.scatter(q, face_to_face_spend[idx], color='blue', zorder=5)

# Customize the plot
plt.title('Indexed Spend Comparison: All vs Online vs F2F')
plt.xlabel('Time Period')
plt.ylabel('Indexed Spend (Base 2019 = 100)')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()

# Save the plot
plt.savefig("indexed_spend_comparison1.png")
plt.show()



In [ ]:
# Growth Rate 2025Q1 to 2019Q1 for Indexed_All_Online_mcg.csv

import pandas as pd

# Load the CSV file
file_path = 'Indexed_All_Online_mcg.csv'
df = pd.read_csv(file_path)

# Extract the values for 2019Q1 and 2025Q1
value_2019Q1 = df.loc[df['time_period_value'] == '2019Q1', 'Indexed Spend Online All'].values[0]
value_2025Q1 = df.loc[df['time_period_value'] == '2025Q1', 'Indexed Spend Online All'].values[0]

# Calculate the percentage growth
growth_rate = ((value_2025Q1 - value_2019Q1) / value_2019Q1) * 100

growth_rate



In [ ]:
# Growth Rate 2025Q1 to 2019Q1 for Indexed_All_F2F.csv

import pandas as pd

# Load the CSV file
file_path = 'Indexed_All_F2F_mcg.csv'
df = pd.read_csv(file_path)

# Extract the values for 2019Q1 and 2025Q1
value_2019Q1 = df.loc[df['time_period_value'] == '2019Q1', 'Indexed Spend F2F All'].values[0]
value_2025Q1 = df.loc[df['time_period_value'] == '2025Q1', 'Indexed Spend F2F All'].values[0]

# Calculate the percentage growth
growth_rate = ((value_2025Q1 - value_2019Q1) / value_2019Q1) * 100

growth_rate

